In [ ]:
import numpy as np
import nengo
import nengolib
from nengolib.signal import s

import matplotlib.pyplot as plt
from nengo.utils.matplotlib import rasterplot
import seaborn as sns
sns.set(context='poster', style='whitegrid')
%matplotlib inline

from nengo_bioneurons import BahlNeuron, build_filter

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [ ]:
def train_filter(spikes,
                 target,
                 method='hyperopt',
                 n_zeros=0,
                 n_poles=2,
                 z_min=0,
                 z_max=1,
                 p_min=-1e2,
                 p_max=-1e1,
                 reg_min=1e-1,
                 reg_max=1e-0,
                 max_evals=10,
                 hs=3,  # hyperopt seed
                 ):
    
    if method == 'hyperopt':
#         print 'Running hyperopt ...'
        hyperparams = {}
        for z in range(n_zeros):
            hyperparams['zeros_%s'%z] = hp.uniform('zeros_%s'%z, z_min, z_max)
        for p in range(n_poles):
            hyperparams['poles_%s'%p] = hp.uniform('poles_%s'%p, p_min, p_max)
        hyperparams['reg'] = hp.uniform('reg', reg_min, reg_max)
        hyperparams['n_zeros'] = n_zeros
        hyperparams['n_poles'] = n_poles
        hyperparams['spikes'] = spikes
        hyperparams['target'] = target

        def objective(hyperparams):
            zeros = np.array([hyperparams['zeros_%s'%z] for z in range(hyperparams['n_zeros'])])
            poles = np.array([hyperparams['poles_%s'%p] for p in range(hyperparams['n_poles'])])
            reg = hyperparams['reg']
            
            h = nengolib.signal.LinearSystem((zeros, poles, 1.0))
            h /= h.dcgain
            
            act = h.filt(np.load(hyperparams['spikes'])['spikes'])
            target = np.load(hyperparams['target'])['target']
            d = nengo.solvers.LstsqL2(reg=reg)(act, target)[0]
            xhat = np.dot(act, d)
            nrmse = nengolib.signal.nrmse(xhat, target=target)
            
            return {'loss': nrmse,
                    'zeros': zeros,
                    'poles': poles,
                    'reg': reg,
                    'status': STATUS_OK }
        
        trials = Trials()

        best = fmin(objective,
                    rstate=np.random.RandomState(seed=hs),
                    space=hyperparams,
                    algo=tpe.suggest,
                    max_evals=max_evals,
                    trials=trials)
        
        best_zeros = []
        best_poles = []
        for z in range(n_zeros):
            best_zeros.append(best['zeros_%s'%z])
        for p in range(n_poles):
            best_poles.append(best['poles_%s'%p])
        best_reg = best['reg']
        
        return best_zeros, best_poles, best_reg

### Demo with LIF() neurons

In [ ]:
# with nengo.Network() as network:
#     stim = nengo.Node(lambda t: np.sin(t))
#     ens = nengo.Ensemble(20, 1)
#     nengo.Connection(stim, ens)
#     p = nengo.Probe(ens.neurons, 'spikes')
#     target = nengo.Probe(stim)
    
# with nengo.Simulator(network) as sim:
#     sim.run(4*np.pi)
    
# spikes = sim.data[p]
# target = sim.data[target]
# np.savez('data/spikes.npz', spikes=spikes)
# np.savez('data/target.npz', target=target)

# best_zeros, best_poles, best_reg = train_filter(
#     'data/spikes.npz',
#     'data/target.npz',
#     max_evals=100)
# h = nengolib.signal.LinearSystem((best_zeros, best_poles, 1.0))
# h /= h.dcgain
# solver = nengo.solvers.LstsqL2(reg=best_reg)

# act = h.filt(spikes)
# d = solver(act, target)[0]
# xhat = np.dot(act, d)
# nrmse = nengolib.signal.nrmse(xhat, target=target)

# fig, ax = plt.subplots(1, 1)
# ax.plot(sim.trange(), xhat, label='$\hat{x}$')
# ax.plot(sim.trange(), target, label='target')
# ax.set(xlabel='time', ylabel='$x$', title='nrmse=%.3f'%nrmse)
# ax.legend()
# plt.show()

For the simplest training, I'll use one regularization value across sections.
1. Optimize that value for the inter_bio connection
  - fix the synaptic filter to tau
2. Optimize a regularization value for the bioneuron readout at each transition point.
  - use the optimized zeros, poles, and reg to calculate d_out 
3. Set d_bio_bio = d_out for the next transition point
  - fix the bio_bio connection to ExpSyn(tau)
  - optimized d_out may perform poorly when using ExpSyn(tau) instead of optimized zeros/poles

In [ ]:
def train(transition=[0, 1],
       n=100,
       signal='cos',
       es=0,
       cs=1,
       ss=2,
       hs=3, 
       tau=0.1,
       tau_readout=0.1,
       freq=2*np.pi,
       t=1,
       verbose=False,
       n_syns=[1],
       secs=['apical'],
       n_zeros=0,
       n_poles=1,
       p_min=-1.0001/0.1, # coresponds to lowpass(tau=0.1)
       p_max=-1.0/0.1,
       reg_min=1e-1,  # nengo default
       reg_max=1.0001e-1,
       max_evals=100,
       neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.01)):
 
    # first find the norm of the filtered signal
    lpf=nengo.Lowpass(tau)
    lpf_readout=nengo.Lowpass(tau_readout)
    with nengo.Network() as model:
        if signal == 'cos':
            stim = nengo.Node(output=lambda t: np.cos(freq*t))
        elif signal == 'sin':
            stim = nengo.Node(output=lambda t: np.sin(freq*t))
        elif signal == 'white_noise':
            stim = nengo.Node(nengo.processes.WhiteSignal(
                period=100,
                high=freq,
                rms=0.5,
                seed=ss))
        p_stim = nengo.Probe(stim, synapse=None)
        p_integral = nengo.Probe(stim, synapse=1/s)
    with nengo.Simulator(model, progress_bar=False) as sim:
        sim.run(t, progress_bar=False)
    stimulus = sim.data[p_stim]
    target = sim.data[p_integral]
    target_f = lpf.filt(sim.data[p_integral])
    target_ff = lpf_readout.filt(lpf.filt(sim.data[p_integral]))
    norm_s = np.max(np.abs(stimulus))
    norm = np.max(np.abs(target))
    norm_f = np.max(np.abs(target_f))
    norm_ff = np.max(np.abs(target_ff))
    
    # now solve for "inter" decoders, which will be used to compute
    # weights on the inter_bio connection
    with nengo.Network() as model:
        if signal == 'cos':
            stim = nengo.Node(output=lambda t: np.cos(freq*t))
        elif signal == 'sin':
            stim = nengo.Node(output=lambda t: np.sin(freq*t))
        elif signal == 'white_noise':
            stim = nengo.Node(nengo.processes.WhiteSignal(
                period=100,
                high=freq,
                rms=0.5,
                seed=ss))
        pre2 = nengo.Ensemble(100, 1, radius=norm, neuron_type=nengo.LIF(), seed=es)
        inter = nengo.Ensemble(n, 1, neuron_type=neuron_type, seed=es)
        tar = nengo.Ensemble(1, 1, neuron_type=nengo.Direct())       
        nengo.Connection(stim, pre2, synapse=1/s)
        for j, sec in enumerate(secs):
            nengo.Connection(pre2, inter,
                             sec=sec,
                             n_syn=n_syns[j],
                             syn_type='ExpSyn',
                             tau_list=[0.01],
                             synapse=0.01,
                             transform=1.0/norm_f,
                             seed=cs)
        nengo.Connection(stim, tar, synapse=1/s, transform=1.0/norm_f)
        p_a = nengo.Probe(inter.neurons, synapse=None)
        p_integral = nengo.Probe(tar, synapse=None)  

    with nengo.Simulator(model, progress_bar=verbose) as sim:
        sim.run(t, progress_bar=verbose)

    spikes = sim.data[p_a]
    target_f = lpf_readout.filt(sim.data[p_integral])
    target_ff = lpf_readout.filt(lpf.filt(sim.data[p_integral]))
    np.savez('data/spikes.npz', spikes=spikes)
    np.savez('data/target.npz', target=target_f)
    
    best_zeros, best_poles, best_reg = train_filter(
        'data/spikes.npz',
        'data/target.npz',
        max_evals=max_evals,
        n_zeros=0,  # needs to match synapse on bio_bio  # n_zeros,
        n_poles=1,  # needs to match synapse on bio_bio, # n_poles,
        p_min=-1.0001/tau,
        p_max=-1.0/tau,
        reg_min=reg_min,
        reg_max=reg_max,
        hs=hs)

    h = nengolib.signal.LinearSystem((best_zeros, best_poles, 1.0))
    h /= h.dcgain
    solver = nengo.solvers.LstsqL2(reg=best_reg)
    act = h.filt(spikes)
    d_inter = solver(act, target_f)[0]
    xhat = np.dot(act, d_inter)
    nrmse = nengolib.signal.nrmse(xhat, target=target_f)

#     print 'best_reg=%.3f (inter)' %(best_reg)
    if verbose:
        fig, (ax, ax2, ax3) = plt.subplots(1, 3, figsize=((12, 4)))
        rasterplot(sim.trange(), spikes, ax=ax)
        ax.set(xlabel='time', ylabel='neuron')
        times = np.arange(0, 1e0, 1e-3)
        ax2.plot(times, h.impulse(len(times)), label='optimized')
        ax2.plot(times, nengolib.Lowpass(tau).impulse(len(times)), label='lowpass')
        ax2.legend()
        ax2.set(xlabel='time', ylabel='amplitude', title=r'$reg=%.3f$'%best_reg)
        ax3.plot(sim.trange(), target_f, label=r'$x_f$', linestyle='--')
#         ax.plot(sim.trange(), target_ff, label=r'$x_{ff}$', linestyle='--')
        ax3.plot(sim.trange(), xhat, label=r'$\hat{x}$')
        ax3.set(xlabel='time', ylabel=r'$x$', title=r'inter, nrmse=%.3f'%nrmse)
        ax3.legend()
        plt.tight_layout()
        plt.show()

        
    # Transition from the supervised (inter spikes) to the unsupervised
    # network, updating h_out and d_out at each transition point
    # using train_filter
    d_bios = [np.zeros((n, 1)) for _ in secs]
    d_inters = [d_inter for _ in secs]
    nrmses = []
    for p in transition:
        print "p=%.3f"%p
        with nengo.Network() as model:
            if signal == 'cos':
                stim = nengo.Node(output=lambda t: np.cos(freq*t))
            elif signal == 'sin':
                stim = nengo.Node(output=lambda t: np.sin(freq*t))
            elif signal == 'white_noise':
                stim = nengo.Node(nengo.processes.WhiteSignal(
                    period=100,
                    high=freq,
                    rms=0.5,
                    seed=ss))
            pre = nengo.Ensemble(n, 1, radius=norm_s, neuron_type=nengo.LIF(), seed=es)
            pre2 = nengo.Ensemble(100, 1, radius=norm, neuron_type=nengo.LIF(), seed=es)
            inter = nengo.Ensemble(n, 1, neuron_type=neuron_type, seed=es)
            bio = nengo.Ensemble(n, 1, neuron_type=neuron_type, seed=es)
            tar = nengo.Ensemble(1, 1, neuron_type=nengo.Direct(), seed=es)
            
            nengo.Connection(stim, pre, synapse=None)
            nengo.Connection(stim, pre2, synapse=1/s)

            for j, sec in enumerate(secs):
                nengo.Connection(pre, bio,
                                 sec=sec,
                                 n_syn=n_syns[j],
                                 syn_type='ExpSyn',
                                 tau_list=[tau],
                                 synapse=tau,
                                 transform=tau/norm_f,
                                 seed=cs)
                nengo.Connection(pre2, inter,
                                 sec=sec,
                                 n_syn=n_syns[j],
                                 syn_type='ExpSyn',
                                 tau_list=[0.01],
                                 synapse=0.01,
                                 transform=1.0/norm_f,
                                 seed=cs)
                nengo.Connection(bio, bio,
                                 sec=sec,
                                 n_syn=n_syns[j],
                                 syn_type='ExpSyn',
                                 tau_list=[tau],
                                 synapse=tau,
                                 transform=p,
                                 solver=nengo.solvers.NoSolver(d_bios[j]),
                                 seed=cs)
                nengo.Connection(inter, bio,
                                 sec=sec,
                                 n_syn=n_syns[j],
                                 syn_type='ExpSyn',
                                 tau_list=[tau],
                                 synapse=tau,
                                 transform=1-p,
                                 solver=nengo.solvers.NoSolver(d_inters[j]),
                                 seed=cs)
            
            nengo.Connection(stim, tar, synapse=1/s, transform=1.0/norm_f)
            
            p_a = nengo.Probe(bio.neurons, synapse=None)
            p_inter = nengo.Probe(inter, synapse=tau_readout, solver=nengo.solvers.NoSolver(d_inter))
            p_integral = nengo.Probe(tar, synapse=None)
            
        with nengo.Simulator(model, progress_bar=verbose) as sim:
            sim.run(t, progress_bar=verbose)
        
        spikes = sim.data[p_a]
        target_f = lpf_readout.filt(sim.data[p_integral])
        target_ff = lpf_readout.filt(lpf.filt(sim.data[p_integral]))
        np.savez('data/spikes.npz', spikes=spikes)
        np.savez('data/target.npz', target=target_ff)
        
        best_zeros, best_poles, best_reg = train_filter(
            'data/spikes.npz',
            'data/target.npz',
            max_evals=100,
            n_zeros=n_zeros,
            n_poles=n_poles,
            p_min=p_min,
            p_max=p_max,
            reg_min=reg_min,
            reg_max=reg_max)

        h = nengolib.signal.LinearSystem((best_zeros, best_poles, 1.0))
        h /= h.dcgain
        solver = nengo.solvers.LstsqL2(reg=best_reg)
        act = h.filt(spikes)
        d_bio = solver(act, target_f)[0]
        xhat = np.dot(act, d_bio)
        nrmse = nengolib.signal.nrmse(xhat, target=target_f)
        nrmses.append(nrmse)
        d_bios = [d_bio for _ in secs]

#         print 'best_reg=%.3f (p=%s)' %(best_reg, p)
        if verbose:
            fig, (ax, ax2, ax3) = plt.subplots(1, 3, figsize=((12, 4)))
            rasterplot(sim.trange(), spikes, ax=ax)
            ax.set(xlabel='time', ylabel='neuron')
            times = np.arange(0, 1e0, 1e-3)
            ax2.plot(times, h.impulse(len(times)), label='optimized')
            ax2.plot(times, nengolib.Lowpass(tau).impulse(len(times)), label='lowpass')
            ax2.legend()
            ax2.set(xlabel='time', ylabel='amplitude', title=r'$reg=%.3f$'%best_reg)
            ax3.plot(sim.trange(), target_f, label=r'$x_f$', linestyle='--')
    #         ax.plot(sim.trange(), target_ff, label=r'$x_{ff}$', linestyle='--')
            ax3.plot(sim.trange(), xhat, label=r'$\hat{x}$')
            ax3.set(xlabel='time', ylabel=r'$x$', title=r'bio, p=%.3f, nrmse=%.3f'%(p, nrmse))
            ax3.legend()
            plt.tight_layout()
            plt.show()
            
    fig, ax = plt.subplots(1, 1)
    ax.plot(np.arange(0, len(transition)), nrmses)
    ax.set(xlabel='iteration', ylabel='nrmse')
    plt.show()
        
    return d_inters, d_bios, h

In [ ]:
def test(d_bios,
       h,
       n=100,
       signal='cos',
       es=0,
       cs=1,
       ss=2,
       tau=0.1,
       tau_readout=0.1,
       freq=2*np.pi,
       t=1,
       verbose=False,
       n_syns=[1],
       secs=['apical'],
       neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.01)):
 
    # first find the norm of the filtered signal
    lpf=nengo.Lowpass(tau)
    lpf_readout=nengo.Lowpass(tau_readout)
    with nengo.Network() as model:
        if signal == 'cos':
            stim = nengo.Node(output=lambda t: np.cos(freq*t))
        elif signal == 'sin':
            stim = nengo.Node(output=lambda t: np.sin(freq*t))
        elif signal == 'white_noise':
            stim = nengo.Node(nengo.processes.WhiteSignal(
                period=100,
                high=freq,
                rms=0.5,
                seed=ss))
        p_stim = nengo.Probe(stim, synapse=None)
        p_integral = nengo.Probe(stim, synapse=1/s)
    with nengo.Simulator(model, progress_bar=False) as sim:
        sim.run(t, progress_bar=False)
    stimulus = sim.data[p_stim]
    target = sim.data[p_integral]
    target_f = lpf.filt(sim.data[p_integral])
    target_ff = lpf_readout.filt(lpf.filt(sim.data[p_integral]))
    norm_s = np.max(np.abs(stimulus))
    norm = np.max(np.abs(target))
    norm_f = np.max(np.abs(target_f))
    norm_ff = np.max(np.abs(target_ff))
    
    with nengo.Network() as model:
        if signal == 'cos':
            stim = nengo.Node(output=lambda t: np.cos(freq*t))
        elif signal == 'sin':
            stim = nengo.Node(output=lambda t: np.sin(freq*t))
        elif signal == 'white_noise':
            stim = nengo.Node(nengo.processes.WhiteSignal(
                period=100,
                high=freq,
                rms=0.5,
                seed=ss))
        pre = nengo.Ensemble(n, 1, radius=norm_s, neuron_type=nengo.LIF(), seed=es)
        bio = nengo.Ensemble(n, 1, neuron_type=neuron_type, seed=es)
        lif = nengo.Ensemble(n, 1, neuron_type=nengo.LIF(), seed=es)
        tar = nengo.Ensemble(1, 1, neuron_type=nengo.Direct(), seed=es)

        nengo.Connection(stim, pre, synapse=None)

        for j, sec in enumerate(secs):
            nengo.Connection(pre, bio,
                             sec=sec,
                             n_syn=n_syns[j],
                             syn_type='ExpSyn',
                             tau_list=[tau],
                             synapse=tau,
                             transform=tau/norm_f,
                             seed=cs)
            nengo.Connection(bio, bio,
                             sec=sec,
                             n_syn=n_syns[j],
                             syn_type='ExpSyn',
                             tau_list=[tau],
                             synapse=tau,
                             solver=nengo.solvers.NoSolver(d_bios[j]),
                             seed=cs)
        nengo.Connection(pre, lif, synapse=tau, transform=tau/norm_f, seed=cs)
        nengo.Connection(lif, lif, synapse=tau, seed=cs)
        nengo.Connection(stim, tar, synapse=1/s, transform=1.0/norm_f)

        p_a = nengo.Probe(bio.neurons, synapse=None)
        p_bio = nengo.Probe(bio, synapse=h, solver=nengo.solvers.NoSolver(d_bios[0]))
        p_lif = nengo.Probe(lif, synapse=tau_readout)
        p_integral = nengo.Probe(tar, synapse=None)

    with nengo.Simulator(model, progress_bar=verbose) as sim:
        sim.run(t, progress_bar=verbose)

    spikes = sim.data[p_a]
#     act = h.filt(spikes)
    target_f = lpf_readout.filt(sim.data[p_integral])
    target_ff = lpf_readout.filt(lpf.filt(sim.data[p_integral]))
#     xhat = np.dot(act, d_bios[0])
    xhat = sim.data[p_bio]
    xhat_lif = sim.data[p_lif]
    nrmse = nengolib.signal.nrmse(xhat, target=target_f)
    nrmse_lif = nengolib.signal.nrmse(xhat_lif, target=target_f)

    if verbose:
        fig, (ax, ax2, ax3) = plt.subplots(1, 3, figsize=((12, 4)))
        rasterplot(sim.trange(), spikes, ax=ax)
        ax.set(xlabel='time', ylabel='neuron')
        times = np.arange(0, 1e0, 1e-3)
        ax2.plot(times, h.impulse(len(times)), label='optimized')
        ax2.plot(times, nengolib.Lowpass(tau).impulse(len(times)), label='lowpass')
        ax2.legend()
        ax2.set(xlabel='time', ylabel='amplitude')
        ax3.plot(sim.trange(), target_f, label=r'$x_f$', linestyle='--')
#         ax.plot(sim.trange(), target_ff, label=r'$x_{ff}$', linestyle='--')
        ax3.plot(sim.trange(), xhat, label=r'$\hat{x}$, nrmse=%.3f'%nrmse)
        ax3.plot(sim.trange(), xhat_lif, label='$\hat{x}_{lif}$\n nrmse=%.3f'%nrmse_lif)
        ax3.set(xlabel='time', ylabel=r'$x$')
        ax3.legend()
        plt.tight_layout()
        plt.show()

# ALIF Experiments

### influence of regularization ranges
 - does the optimizer always choose the lowest reg values for the training set?
 - do these perform systematically different on the test set?
 - does Aaron's result with decoder divergence emerge?

## ALIF()

$reg=0.1$ (default), max_evals=1 (no filter optimization)

In [ ]:
d_inters, d_bios, h = train(
    t=2,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.01),
    verbose=True,
    reg_min=1e-1,
    reg_max=1.001e-1,
    p_min=-1.0001/0.1,
    p_max=-1.0/0.1,
#     n_poles=1,
    max_evals=1,
    freq=2*np.pi)

test(d_bios, h, t=2.0, verbose=True, signal='cos', freq=2*np.pi)

test(d_bios, h, t=8.0, verbose=True, signal='white_noise', freq=2*np.pi)

$reg=1.0$ (high)

In [ ]:
d_inters, d_bios, h = train(
    t=2,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.01),
    verbose=True,
    reg_min=1e0,
    reg_max=1.001e0,
    max_evals=1,
    freq=2*np.pi)

test(d_bios, h, t=2.0, verbose=True, signal='cos', freq=2*np.pi)

test(d_bios, h, t=8.0, verbose=True, signal='white_noise', freq=2*np.pi)

Lower frequency

In [ ]:
d_inters, d_bios, h = train(
    t=8,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.01),
    verbose=True,
    reg_min=1e-1,
    reg_max=1.001e-1,
    max_evals=1,
    freq=0.5*np.pi)

test(d_bios, h, t=8.0, verbose=True, signal='cos', freq=0.5*np.pi)

test(d_bios, h, t=8.0, verbose=True, signal='white_noise', freq=0.5*np.pi)

Transition

In [ ]:
d_inters, d_bios, h = train(
    transition=np.concatenate((np.linspace(0, 1, 10), np.repeat(1, 10))),
    t=2,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.01),
    verbose=True,
    reg_min=1e-1,
    reg_max=1.001e-1,
    max_evals=1,
    freq=2*np.pi)

test(d_bios,
     h,
     neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.01),
     t=2.0, 
     verbose=True,
     signal='cos',
     freq=2*np.pi)

test(d_bios,
     h,
     neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.01),
     t=8.0,
     verbose=True,
     signal='white_noise',
     freq=2*np.pi)

$reg=1.0$

In [ ]:
d_inters, d_bios, h = train(
    transition=np.concatenate((np.linspace(0, 1, 10), np.repeat(1, 10))),
    t=2,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.01),
    verbose=True,
    reg_min=1e0,
    reg_max=1.001e0,
    max_evals=1,
    freq=2*np.pi)

test(d_bios,
     h,
     neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.01),
     t=2.0, 
     verbose=True,
     signal='cos',
     freq=2*np.pi)

test(d_bios,
     h,
     neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.01),
     t=8.0,
     verbose=True,
     signal='white_noise',
     freq=2*np.pi)

LIF() neurons, $reg=0.1$

In [ ]:
d_inters, d_bios, h = train(
    transition=np.concatenate((np.linspace(0, 1, 10), np.repeat(1, 10))),
    t=2,
    neuron_type=nengo.LIF(),
    verbose=True,
    reg_min=1e-1,
    reg_max=1.001e-1,
    max_evals=1,
    freq=2*np.pi)

test(d_bios,
     h,
     neuron_type=nengo.LIF(),
     t=2.0, 
     verbose=True,
     signal='cos',
     freq=2*np.pi)

test(d_bios,
     h,
     neuron_type=nengo.LIF(),
     t=8.0,
     verbose=True,
     signal='white_noise',
     freq=2*np.pi)

More adaptive ALIF()

In [ ]:
d_inters, d_bios, h = train(
    transition=np.concatenate((np.linspace(0, 1, 10), np.repeat(1, 10))),
    t=2,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.05),
    verbose=True,
    reg_min=1e-1,
    reg_max=1.001e-1,
    max_evals=1,
    freq=2*np.pi)

test(d_bios,
     h,
     neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.05),
     t=2.0, 
     verbose=True,
     signal='cos',
     freq=2*np.pi)

test(d_bios,
     h,
     neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.05),
     t=8.0,
     verbose=True,
     signal='white_noise',
     freq=2*np.pi)

In [ ]:
d_inters, d_bios, h = train(
    transition=np.concatenate((np.linspace(0, 1, 10), np.repeat(1, 30))),
    t=2,
    neuron_type=nengo.LIF(),
    verbose=False,
    reg_min=1e0,
    reg_max=1.001e0,
    max_evals=1,
    freq=2*np.pi)

test(d_bios,
     h,
     neuron_type=nengo.LIF(),
     t=2.0, 
     verbose=True,
     signal='cos',
     freq=2*np.pi)

test(d_bios,
     h,
     neuron_type=nengo.LIF(),
     t=8.0,
     verbose=True,
     signal='white_noise',
     freq=2*np.pi)

# Bioneuron Testing

1. Does the training work with transition=[0,1]?
2. Does increasing n_transition improve final fit?
    - need to collect statistics of white_noise simulations to identify trends
3. Are reg and freq as influential as for the ALIF()?

In [ ]:
d_inters, d_bios, h = train(
    t=3,
    neuron_type=BahlNeuron(),
    secs=['apical', 'tuft'],
    n_syns=[1, 1],
    verbose=True,
    reg_min=1e0,
    reg_max=1.001e0,
    max_evals=1,
    freq=2*np.pi)

test(d_bios,
     h,
     t=3.0,
     neuron_type=BahlNeuron(),
     secs=['apical', 'tuft'],
     n_syns=[1, 1],
     verbose=True,
     signal='cos',
     freq=2*np.pi)

test(d_bios,
     h,
     t=3.0,
     verbose=True,
     neuron_type=BahlNeuron(),
     secs=['apical', 'tuft'],
     n_syns=[1, 1],
     signal='white_noise',
     freq=2*np.pi)

In [ ]:
d_inters, d_bios, h = train(
    t=3,
    neuron_type=BahlNeuron(),
    secs=['apical', 'tuft'],
    n_syns=[1, 1],
    n_poles=2,
    p_min=-1e2,
    p_max=-1e0,
    verbose=True,
    reg_min=1e0,
    reg_max=1.001e0,
    max_evals=100,
    freq=2*np.pi)

test(d_bios,
     h,
     t=3.0,
     neuron_type=BahlNeuron(),
     secs=['apical', 'tuft'],
     n_syns=[1, 1],
     verbose=True,
     signal='cos',
     freq=2*np.pi)

test(d_bios,
     h,
     t=3.0,
     verbose=True,
     neuron_type=BahlNeuron(),
     secs=['apical', 'tuft'],
     n_syns=[1, 1],
     signal='white_noise',
     freq=2*np.pi)